In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pickle
from datetime import datetime
import os
import time
import random

## Save list_movie_data in with pickle
def save_data(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

## Load pickle file data 
def load_data(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
    
#function for scraping infobox
def scrape(url):
    page = requests.get(url)
    toScrape = bs(page.content, 'html.parser')
    movie_details = toScrape.find(class_='infobox vevent')
    rows = movie_details.find_all('tr') 
    movie_data = {}

    title = rows[0].find('th').get_text()
    movie_data['Title'] = title
    for i, row in enumerate(rows):
        try:
            if i <= 1:
                continue
            elif row.find('th').get_text() == 'Based on':
                continue
            elif row.find('th').get_text() == 'Starring':
                clean_tags(row)
                movie_data['Starring'] = clean(row)
                movie_data['Lead'] = movie_data['Starring'][0]
            elif 'Production' in row.find('th').get_text():
                clean_tags(row)
                movie_data['Production companies'] = clean(row)
            elif row.find('th').get_text() == 'Running time':
                clean_tags(row)
                movie_data['Running_time_min'] = clean(row)
            elif row.find('th').get_text() == 'Release date':
                clean_tags(row)
                date = clean(row).strip()
                dt = dt_conversion(date)
                movie_data['Release_date_dt'] = dt
                movie_data['Release_month'] = dt.month
            elif row.find('th').get_text() == 'Budget':
                budget = money_convert(row)
                movie_data['Budget'] = budget
            elif row.find('th').get_text() == 'Box office':
                money = money_convert(row)
                movie_data['Box_office'] = money
            else:
                clean_tags(row)
                column = row.find('th').get_text(' ', strip=True)
                data = clean(row)
                movie_data[column] = data
        except:
            pass
        
    op_rating = get_op_and_rating(title)
    movie_data['Box_office_opening'] = op_rating[0]
    movie_data['Rating'] = op_rating[1]
        
    return movie_data
    
    
    
#grab opening box office numbers and MPAA rating from Box Office Mojo given movie title
def get_op_and_rating(title):
    search_page = requests.get('https://www.boxofficemojo.com/search/?q='+title)
    search_page_content = bs(search_page.content, 'html.parser')
    search_list = search_page_content.find_all('a')
    path = ''
    for li in search_list:
        if li.get_text() == title:
            path = li['href']
            break
        elif title[:10] in li.get_text():
            path = li['href']
            break
            
    if path == '':
        path = search_list[14]['href']
    
    
    data = [None,'Unknown']
    movie_page = requests.get('https://www.boxofficemojo.com'+path)
    movie_page_content = bs(movie_page.content, 'html.parser')
    table_links = movie_page_content.find_all('a')
    for li in table_links:
        if '$' in li.get_text():
            opening_box = float(li.get_text().replace('$', '').replace(',', ''))
            data[0] = opening_box
            break
            
    spans = movie_page_content.find_all('span')
    for span in spans:
        if 'MPAA' in span.get_text():
            rating = span.find_next('span').get_text()
            data[1] = rating
            break
    return data  


#Convert date str to datetime object
def dt_conversion(date):
    patterns = ['%B %d, %Y', '%d %B %Y']
    for pat in patterns:
        try:
            return datetime.strptime(date, pat)
        except:
            pass
    return none


#remove troublesome tags
def clean_tags(content):
    t = ['sup', 'span']
    tags = content.find_all(t)
    for tag in tags:
        tag.decompose()

def money_convert(row):
    multiplier = 1
    money_str = row.find('td').get_text().replace('\xa0', ' ')
    if '£' in money_str:
        multiplier = 1.41
    elif '€' in money_str:
        multiplier = 1.21
        
        
    if 'million' in money_str:
        if '(' in money_str:
            money_str = money_str.split('(')[0].replace('$','').replace('£', '')
        if '-' in money_str:
            number = float(money_str.split('-')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**6)
        if '–' in money_str:
            number = float(money_str.split('–')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**6)
        else:
            number = float(money_str.split(' ')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**6)
    elif 'billion' in money_str:
        if '(' in money_str:
            money_str = money_str.split('(')[0].replace('$','').replace('£', '')
        if '-' in money_str:
            number = float(money_str.split('-')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**9)
        if '–' in money_str:
            number = float(money_str.split('–')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**9)
        else:
            number = float(money_str.split(' ')[0].replace('$','').replace('£', ''))
            return number * multiplier * (10**9)
    else:
        number = float(money_str.replace(',','').replace('$','').replace('£', ''))
        return number * multiplier


#function to clean data scraped from wikipedia infobox
def clean(row):
    if row.find('th').get_text() == 'Release date':
        if row.find('td').get_text()[0].isdigit() == True:
            return row.find('td').get_text().split(',')[0].replace('\xa0', ' ').strip('\n').strip(' ')  
        return row.find('td').get_text().split('(')[0].replace('\xa0', ' ').strip('\n').strip(' ')                                                                                         
    elif row.find('th').get_text() == 'Running time':
        return int(row.find('td').get_text().split(' ')[0])
    elif row.find('br'):
        return [text for text in row.find('td').stripped_strings]
    elif row.find('li'):
        return [li.get_text(' ', strip=True).replace('\xa0', ' ') for li in row.find_all('li')]
    return row.find('td').get_text()

# Scrape All Hollywood Feature Films (on wikipedia) from 1990 - 2019

In [3]:
base_url = 'https://en.wikipedia.org' 
list_url = 'https://en.wikipedia.org/wiki/List_of_American_films_of_'
decades = [1990,2020]
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64); tutukito/koolkidsteel8@gmail.com'}
list_movie_data = []

# load in table of Film Eras
for y in range(decades[0],decades[1]):
    year = str(y)
    list_page = requests.get(list_url+year, headers=headers)
    content = bs(list_page.content, 'html.parser')
    table_rows = content.select('.wikitable.sortable i')
    for i,row in enumerate(table_rows):
        print(i)
        start = time.time()
        if i % 10 == 0:
            print(i)
        try:
            path = row.find('a')['href']  
            print(path)
            list_movie_data.append(scrape(base_url+path))
        except: 
            print(path+' : error')
            pass
        delay = time.time() - start
        time.sleep(random.uniform(1,2) * delay)
        
list_movie_data
save_data('1990-2019_movie_data.pickle', list_movie_data)

0
0
/wiki/The_Adventures_of_Ford_Fairlane
1
/wiki/After_Dark,_My_Sweet
2
/wiki/Air_America_(film)
3
/wiki/Alice_(1990_film)
4
/wiki/Almost_an_Angel
5
/wiki/The_Ambulance
6
/wiki/American_Dream_(film)
7
/wiki/American_Ninja_4:_The_Annihilation
8
/wiki/Andre%27s_Mother
/wiki/Andre%27s_Mother : error
9
/wiki/Angel_Town_(film)
10
10
/wiki/Another_48_Hrs.
11
/wiki/48_Hrs.
12
/wiki/Arachnophobia_(film)
13
/wiki/Avalon_(1990_film)
14
/wiki/Awakenings
15
/wiki/Back_to_the_Future_Part_III
16
/wiki/Bad_Influence_(film)
17
/wiki/Basket_Case_2
18
/wiki/Betsy%27s_Wedding
19
/wiki/Bird_on_a_Wire_(film)
20
20
/wiki/Blind_Faith_(1990_film)
21
/wiki/Bloodfist_II
22
/wiki/Blue_Steel_(1990_film)
23
/wiki/The_Bonfire_of_the_Vanities_(film)
24
/wiki/Bride_of_Re-Animator
25
/wiki/Cadence_(film)
26
/wiki/Cadillac_Man
27
/wiki/Camp_Cucamonga
/wiki/Camp_Cucamonga : error
28
/wiki/Captain_America_(1990_film)
29
/wiki/Cartoon_All-Stars_to_the_Rescue
30
30
/wiki/Catchfire
/wiki/Catchfire : error
31
/wiki/Catchfir

/wiki/Somebody_Has_to_Shoot_the_Picture : error
235
/wiki/Soultaker_(film)
/wiki/Soultaker_(film) : error
236
/wiki/Spaced_Invaders
237
/wiki/Spirit_of_%2776_(1990_film)
238
/wiki/Spontaneous_Combustion_(film)
239
/wiki/Spymaker:_The_Secret_Life_of_Ian_Fleming
/wiki/Spymaker:_The_Secret_Life_of_Ian_Fleming : error
240
240
/wiki/Stanley_%26_Iris
241
/wiki/State_of_Grace_(1990_film)
242
/wiki/Stella_(1990_film)
243
/wiki/Strike_It_Rich_(1990_film)
244
/wiki/Taking_Care_of_Business_(film)
245
/wiki/Tales_from_the_Darkside:_The_Movie
246
/wiki/Teenage_Mutant_Ninja_Turtles_(1990_film)
247
/wiki/Terminal_City_Ricochet
/wiki/Terminal_City_Ricochet : error
248
/wiki/Texasville
249
/wiki/The_Last_Picture_Show
250
250
/wiki/That%27s_Black_Entertainment
251
/wiki/That%27s_Entertainment!
252
/wiki/Three_Men_and_a_Little_Lady
253
/wiki/Three_Men_and_a_Baby
254
/wiki/Till_There_Was_You_(1990_film)
255
/wiki/Time_of_Love
256
/wiki/Too_Much_Sun
257
/wiki/To_Sleep_with_Anger
258
/wiki/Total_Recall_(199

181
/wiki/Sgt._Kabukiman_N.Y.P.D.
/wiki/Sgt._Kabukiman_N.Y.P.D. : error
182
/wiki/Shadows_and_Fog
183
/wiki/Shattered_(1991_film)
184
/wiki/Shout_(film)
185
/wiki/Showdown_in_Little_Tokyo
186
/wiki/The_Silence_of_the_Lambs_(film)
187
/wiki/Slacker_(film)
188
/wiki/Sleeping_with_the_Enemy
189
/wiki/Soapdish
190
190
/wiki/Sometimes_They_Come_Back_(film)
191
/wiki/Star_Trek_VI:_The_Undiscovered_Country
192
/wiki/Star_Trek_V:_The_Final_Frontier
193
/wiki/Stepping_Out_(1991_film)
194
/wiki/Stone_Cold_(1991_film)
195
/wiki/Straight_Out_of_Brooklyn
196
/wiki/Strictly_Business_(1991_film)
197
/wiki/Suburban_Commando
198
/wiki/The_Super_(1991_film)
199
/wiki/Switch_(1991_film)
200
200
/wiki/The_Taking_of_Beverly_Hills
201
/wiki/Talent_for_the_Game
202
/wiki/Talkin%27_Dirty_After_Dark
203
/wiki/Teenage_Mutant_Ninja_Turtles_II:_The_Secret_of_the_Ooze
204
/wiki/Teenage_Mutant_Ninja_Turtles_(1990_film)
205
/wiki/Terminator_2:_Judgment_Day
206
/wiki/The_Terminator
207
/wiki/Thelma_%26_Louise
208
/wi

199
/wiki/Tiny_Toon_Adventures:_How_I_Spent_My_Vacation
/wiki/Tiny_Toon_Adventures:_How_I_Spent_My_Vacation : error
200
200
/wiki/To_Grandmother%27s_House_We_Go
/wiki/To_Grandmother%27s_House_We_Go : error
201
/wiki/Tom_and_Jerry:_The_Movie
202
/wiki/Topsy_and_Bunker:_The_Cat_Killers
/wiki/Topsy_and_Bunker:_The_Cat_Killers : error
203
/wiki/Toys_(film)
204
/wiki/Traces_of_Red
205
/wiki/Trespass_(1992_film)
206
/wiki/The_Tune
207
/wiki/The_Turning_(1992_film)
208
/wiki/Twin_Peaks:_Fire_Walk_with_Me
209
/wiki/Under_Siege
210
210
/wiki/Unforgiven
211
/wiki/Universal_Soldier_(1992_film)
212
/wiki/Unlawful_Entry_(film)
213
/wiki/Used_People
214
/wiki/Visions_of_Light
215
/wiki/The_Waterdance
216
/wiki/Waxwork_II:_Lost_in_Time
217
/wiki/Wayne%27s_World_(film)
218
/wiki/Where_the_Day_Takes_You
219
/wiki/Whispers_in_the_Dark_(film)
220
220
/wiki/White_Men_Can%27t_Jump
221
/wiki/White_Sands_(film)
222
/wiki/Who_Do_I_Gotta_Kill%3F
223
/wiki/Wind_(1992_film)
224
/wiki/The_Witness_(short_film)
/wi

206
/wiki/The_Wedding_Banquet
207
/wiki/Weekend_at_Bernie%27s_II
208
/wiki/Weekend_at_Bernie%27s
209
/wiki/What%27s_Eating_Gilbert_Grape
210
210
/wiki/What%27s_Love_Got_to_Do_with_It_(film)
211
/wiki/Where%27s_God_When_I%27m_S-Scared%3F
/wiki/Where%27s_God_When_I%27m_S-Scared%3F : error
212
/wiki/White_Wolves:_A_Cry_in_the_Wild_II
/wiki/White_Wolves:_A_Cry_in_the_Wild_II : error
213
/wiki/Who%27s_the_Man%3F
214
/wiki/Wilder_Napalm
215
/wiki/Woman_of_Desire
216
/wiki/Wrestling_Ernest_Hemingway
217
/wiki/The_Wrong_Man_(1993_film)
218
/wiki/Younger_and_Younger
0
0
/wiki/3_Chains_o%27_Gold_(film)
/wiki/3_Chains_o%27_Gold_(film) : error
1
/wiki/3_Ninjas_Kick_Back
2
/wiki/3_Ninjas_(film)
3
/wiki/8_Seconds
4
/wiki/Above_the_Rim
5
/wiki/Abraham_(1993_film)
6
/wiki/Ace_Ventura:_Pet_Detective
7
/wiki/Africa:_The_Serengeti
8
/wiki/The_Air_Up_There
9
/wiki/Airheads
10
10
/wiki/Aladdin:_The_Return_of_Jafar
11
/wiki/Alien_Nation:_Dark_Horizon
/wiki/Alien_Nation:_Dark_Horizon : error
12
/wiki/Amateur

...

117
/wiki/Bridget_Jones%27s_Baby
118
/wiki/Snowden_(film)
119
/wiki/Blair_Witch_(film)
120
120
/wiki/Hillsong:_Let_Hope_Rise
121
/wiki/The_Magnificent_Seven_(2016_film)
122
/wiki/Storks_(film)
123
/wiki/Miss_Peregrine%27s_Home_for_Peculiar_Children_(film)
124
/wiki/Deepwater_Horizon_(film)
125
/wiki/Queen_of_Katwe
126
/wiki/Masterminds_(2016_film)
127
/wiki/The_Girl_on_the_Train_(2016_film)
128
/wiki/The_Birth_of_a_Nation_(2016_film)
129
/wiki/Middle_School:_The_Worst_Years_of_My_Life_(film)
130
130
/wiki/The_Accountant_(2016_film)
131
/wiki/Kevin_Hart:_What_Now%3F
132
/wiki/Max_Steel_(film)
133
/wiki/Jack_Reacher:_Never_Go_Back
134
/wiki/Keeping_Up_with_the_Joneses_(film)
135
/wiki/Ouija:_Origin_of_Evil
136
/wiki/Boo!_A_Madea_Halloween
137
/wiki/I%27m_Not_Ashamed
138
/wiki/American_Pastoral_(film)
139
/wiki/Inferno_(2016_film)
140
140
/wiki/Doctor_Strange_(2016_film)
141
/wiki/Trolls_(film)
142
/wiki/The_Love_Witch
143
/wiki/Fantastic_Beasts_and_Where_to_Find_Them_(film)
144
/wiki/Moa

217
/wiki/Roman_J._Israel,_Esq.
218
/wiki/Coco_(2017_film)
219
/wiki/Darkest_Hour_(film)
220
220
/wiki/The_Man_Who_Invented_Christmas_(film)
221
/wiki/Call_Me_by_Your_Name_(film)
222
/wiki/The_Disaster_Artist_(film)
223
/wiki/The_Shape_of_Water
224
/wiki/Wonder_Wheel_(film)
225
/wiki/I,_Tonya
226
/wiki/Just_Getting_Started_(film)
227
/wiki/Star_Wars:_The_Last_Jedi
228
/wiki/Ferdinand_(film)
229
/wiki/Beyond_Skyline
230
230
/wiki/Jumanji:_Welcome_to_the_Jungle
231
/wiki/The_Greatest_Showman
232
/wiki/Pitch_Perfect_3
233
/wiki/Downsizing_(film)
234
/wiki/Father_Figures
235
/wiki/The_Post_(film)
236
/wiki/Bright_(film)
237
/wiki/Crooked_House_(film)
238
/wiki/Hostiles_(film)
239
/wiki/All_the_Money_in_the_World
240
240
/wiki/Molly%27s_Game
241
/wiki/Phantom_Thread
0
0
/wiki/Insidious:_The_Last_Key
1
/wiki/The_Strange_Ones
2
/wiki/Stratton_(film)
3
/wiki/Sweet_Country_(2017_film)
4
/wiki/The_Commuter_(film)
5
/wiki/Proud_Mary_(film)
6
/wiki/Acts_of_Violence
7
/wiki/Freak_Show_(film)
8
/wik

235
/wiki/Overlord_(2018_film)
236
/wiki/The_Ballad_of_Buster_Scruggs
237
/wiki/Lazer_Team_2
238
/wiki/Lazer_Team_2 : error
239
/wiki/Fantastic_Beasts:_The_Crimes_of_Grindelwald
240
240
/wiki/Widows_(2018_film)
241
/wiki/Instant_Family
242
/wiki/Ralph_Breaks_the_Internet
243
/wiki/Creed_II
244
/wiki/Green_Book_(film)
245
/wiki/Robin_Hood_(2018_film)
246
/wiki/Roma_(2018_film)
247
/wiki/The_Favourite
248
/wiki/Mowgli:_Legend_of_the_Jungle
249
/wiki/The_Possession_of_Hannah_Grace
250
250
/wiki/Anna_and_the_Apocalypse
251
/wiki/Capernaum_(film)
252
/wiki/Mary_Queen_of_Scots_(2018_film)
253
/wiki/Ben_Is_Back
254
/wiki/Spider-Man:_Into_the_Spider-Verse
255
/wiki/Mortal_Engines_(film)
256
/wiki/The_Mule_(2018_film)
257
/wiki/If_Beale_Street_Could_Talk_(film)
258
/wiki/Mary_Poppins_Returns
259
/wiki/Aquaman_(film)
260
260
/wiki/Bumblebee_(film)
261
/wiki/Welcome_to_Marwen
262
/wiki/Second_Act_(film)
263
/wiki/Cold_War_(2018_film)
264
/wiki/Holmes_%26_Watson
265
/wiki/Vice_(2018_film)
266
/wik

214
/wiki/Waves_(film)
215
/wiki/Frozen_II
216
/wiki/A_Beautiful_Day_in_the_Neighborhood
217
/wiki/21_Bridges
218
/wiki/Dark_Waters_(2019_film)
219
/wiki/Knives_Out
220
220
/wiki/Queen_%26_Slim
221
/wiki/The_Two_Popes
222
/wiki/Playmobil:_The_Movie
223
/wiki/The_Aeronauts_(film)
224
/wiki/A_Million_Little_Pieces_(film)
225
/wiki/Daniel_Isn%27t_Real
226
/wiki/Jumanji:_The_Next_Level
227
/wiki/Richard_Jewell_(film)
228
/wiki/Black_Christmas_(2019_film)
229
/wiki/6_Underground_(film)
/wiki/6_Underground_(film) : error
230
230
/wiki/Bombshell_(2019_film)
231
/wiki/Uncut_Gems
232
/wiki/Seberg
233
/wiki/A_Hidden_Life_(2019_film)
234
/wiki/Star_Wars:_The_Rise_of_Skywalker
235
/wiki/Cats_(2019_film)
236
/wiki/Togo_(film)
237
/wiki/Spies_in_Disguise
238
/wiki/Little_Women_(2019_film)
239
/wiki/1917_(2019_film)
240
240
/wiki/Just_Mercy
241
/wiki/Clemency_(film)


[{'Title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Lead': 'Andrew Dice Clay',
  'Music by': ['Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Production companies': ['Silver Pictures'],
  'Distributed by': '20th Century Fox',
  'Release_date_dt': datetime.datetime(1990, 7, 11, 0, 0),
  'Release_month': 7,
  'Running_time_min': 102,
  'Country': 'United States',
  'Language': 'English',
  'Budget': 20000000.0,
  'Box_office': 21400000.0,
  'Box_office_opening': 6374391.0,
  'Rating': 'Unknown'},
 {'Title': 'After Dark, My Sweet',
  'Directed by': 'James Foley',
  'Produced by': ['Ric Kidney', 'Robert Redlin'],
  'Screenplay by': [

# Pickle data and drop irrelevant columns and na rows

In [13]:
# load movie data 
list_movie_data = load_data('1990-2019_movie_data.pickle')

# create pandas dataframe
df = pd.DataFrame(list_movie_data)
pd.set_option("display.max_columns", None)

# # dropping irrelevant columns
for x in range(123):
    df.drop(df.columns[21], axis=1, inplace = True)

df.drop(df.columns[[3,4,7,9]], axis=1, inplace = True)
df = df.set_index('Title')

df.head()


df.to_csv('../Data/all_movie_data_cleaned.csv')

df1 = df.dropna(subset=['Release_month','Budget', 'Box_office_opening', 'Box_office'])
df1.to_csv('../CleanData/all_movie_data_dropna.csv')